<a href="https://colab.research.google.com/github/putrianisah0112/dashboard-absensi/blob/main/DASHBOARDFIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date


In [3]:
data_siswa = pd.DataFrame({
    "Nama": ["Siswa1", "Siswa2", "Siswa3"],
    "Kelas": ["XI IPA", "XI IPA", "XI IPA"]
})

data_siswa


,Nama,Kelas
0,Siswa1,XI IPA
1,Siswa2,XI IPA
2,Siswa3,XI IPA


In [4]:
absensi_log = pd.DataFrame(columns=["Tanggal", "Nama", "Status"])
absensi_log


,Tanggal,Nama,Status


In [5]:
dropdown_absen = widgets.Dropdown(
    options=data_siswa["Nama"].tolist(),
    description="Siswa:"
)

dropdown_dashboard = widgets.Dropdown(
    options=data_siswa["Nama"].tolist(),
    description="Pilih Siswa:"
)


In [6]:
nama_siswa_baru = widgets.Text(description="Nama:")
kelas_siswa_baru = widgets.Text(description="Kelas:")

btn_tambah_siswa = widgets.Button(description="Tambah Siswa", button_style="info")
output_tambah = widgets.Output()

display(nama_siswa_baru, kelas_siswa_baru, btn_tambah_siswa, output_tambah)


Text(value='', description='Nama:')

Text(value='', description='Kelas:')

Button(button_style='info', description='Tambah Siswa', style=ButtonStyle())

Output()

In [7]:
def tambah_siswa(b):
    global data_siswa

    if nama_siswa_baru.value == "" or kelas_siswa_baru.value == "":
        with output_tambah:
            clear_output()
            print("❌ Nama dan kelas harus diisi")
        return

    data_baru = {
        "Nama": nama_siswa_baru.value,
        "Kelas": kelas_siswa_baru.value
    }

    data_siswa = pd.concat([data_siswa, pd.DataFrame([data_baru])], ignore_index=True)

    dropdown_absen.options = data_siswa["Nama"].tolist()
    dropdown_dashboard.options = data_siswa["Nama"].tolist()

    with output_tambah:
        clear_output()
        print("✅ Siswa berhasil ditambahkan")
        display(data_siswa)

    nama_siswa_baru.value = ""
    kelas_siswa_baru.value = ""

btn_tambah_siswa.on_click(tambah_siswa)


In [8]:
tanggal_input = widgets.DatePicker(description="Tanggal:", value=date.today())

status_input = widgets.Dropdown(
    options=["Hadir", "Izin", "Alpha"],
    description="Status:"
)

btn_simpan_absen = widgets.Button(description="Simpan Absensi", button_style="success")
output_absen = widgets.Output()

display(tanggal_input, dropdown_absen, status_input, btn_simpan_absen, output_absen)


DatePicker(value=datetime.date(2026, 1, 30), description='Tanggal:')

Dropdown(description='Siswa:', options=('Siswa1', 'Siswa2', 'Siswa3', 'SISWA 4'), value='Siswa1')

Dropdown(description='Status:', options=('Hadir', 'Izin', 'Alpha'), value='Hadir')

Button(button_style='success', description='Simpan Absensi', style=ButtonStyle())

Output()

In [9]:
def simpan_absensi(b):
    global absensi_log

    data_baru = {
        "Tanggal": tanggal_input.value.strftime("%Y-%m-%d"),
        "Nama": dropdown_absen.value,
        "Status": status_input.value
    }

    absensi_log = pd.concat([absensi_log, pd.DataFrame([data_baru])], ignore_index=True)

    with output_absen:
        clear_output()
        print("✅ Absensi tersimpan")
        display(absensi_log)

btn_simpan_absen.on_click(simpan_absensi)


In [10]:
def hitung_rekap_absensi():
    if absensi_log.empty:
        return pd.DataFrame(columns=["Nama", "Persentase Kehadiran (%)"])

    total_pertemuan = absensi_log["Tanggal"].nunique()

    hadir = absensi_log[absensi_log["Status"] == "Hadir"]
    jumlah_hadir = hadir.groupby("Nama").size()

    persentase = (jumlah_hadir / total_pertemuan * 100).round(2)

    df = persentase.reset_index()
    df.columns = ["Nama", "Persentase Kehadiran (%)"]

    return df


In [11]:
output_dashboard = widgets.Output()

def tampilkan_dashboard():
    with output_dashboard:
        clear_output()

        siswa = dropdown_dashboard.value

        print(f"📊 Dashboard {siswa}")
        print("-"*30)

        rekap = hitung_rekap_absensi()
        display(rekap[rekap["Nama"] == siswa])

display(dropdown_dashboard, output_dashboard)
tampilkan_dashboard()


Dropdown(description='Pilih Siswa:', options=('Siswa1', 'Siswa2', 'Siswa3', 'SISWA 4'), value='Siswa1')

Output()

In [12]:
def on_change(change):
    tampilkan_dashboard()

dropdown_dashboard.observe(on_change, names='value')


In [13]:
btn_simpan_data = widgets.Button(description="💾 Simpan Data", button_style="success")
btn_load_data = widgets.Button(description="📂 Load Data", button_style="warning")
output_save = widgets.Output()

display(btn_simpan_data, btn_load_data, output_save)


Button(button_style='success', description='💾 Simpan Data', style=ButtonStyle())

Button(button_style='warning', description='📂 Load Data', style=ButtonStyle())

Output()

In [14]:
def simpan_data(b):
    global data_siswa, absensi_log

    data_siswa.to_csv("data_siswa.csv", index=False)
    absensi_log.to_csv("absensi_log.csv", index=False)

    with output_save:
        clear_output()
        print("✅ Data berhasil disimpan ke file CSV")

btn_simpan_data.on_click(simpan_data)


In [15]:
def load_data(b):
    global data_siswa, absensi_log

    try:
        data_siswa = pd.read_csv("data_siswa.csv")
        absensi_log = pd.read_csv("absensi_log.csv")

        # update dropdown
        dropdown_absen.options = data_siswa["Nama"].tolist()
        dropdown_dashboard.options = data_siswa["Nama"].tolist()

        with output_save:
            clear_output()
            print("✅ Data berhasil di-load")
            display(data_siswa)
            display(absensi_log)

    except:
        with output_save:
            clear_output()
            print("❌ File belum ada. Simpan data dulu.")

btn_load_data.on_click(load_data)


In [16]:
import matplotlib.pyplot as plt


In [17]:
def tampilkan_grafik_kehadiran():
    df = hitung_rekap_absensi()

    if df.empty:
        print("Belum ada data absensi")
        return

    plt.figure()
    plt.bar(df["Nama"], df["Persentase Kehadiran (%)"])
    plt.xlabel("Nama Siswa")
    plt.ylabel("Persentase Kehadiran (%)")
    plt.title("Grafik Kehadiran Siswa")
    plt.ylim(0,100)
    plt.show()


In [18]:
btn_grafik = widgets.Button(description="📊 Tampilkan Grafik", button_style="info")
output_grafik = widgets.Output()

display(btn_grafik, output_grafik)

def klik_grafik(b):
    with output_grafik:
        clear_output()
        tampilkan_grafik_kehadiran()

btn_grafik.on_click(klik_grafik)


Button(button_style='info', description='📊 Tampilkan Grafik', style=ButtonStyle())

Output()

In [20]:
%%writefile app.py
# paste seluruh kode Streamlit kamu di bawah sini


Writing app.py


In [ ]:
%%writefile requirements.txt
streamlit
pandas
matplotlib
